## Neo4j Import of GraphRAG Result Parquet files
This notebook imports the results of the GraphRAG indexing process into the Neo4j Graph database for further processing, analysis or visualization.

### How does it work?
The notebook loads the parquet files from the output folder of your indexing process and loads them into Pandas dataframes. It then uses a batching approach to send a slice of the data into Neo4j to create nodes and relationships and add relevant properties. The id-arrays on most entities are turned into relationships.

All operations use `MERGE`, so they are idempotent, and you can run the script multiple times.

If you need to clean out the database, you can run the following statement
```
MATCH (n)
CALL { WITH n DETACH DELETE n } IN TRANSACTIONS OF 25000 ROWS;
```

In [1]:
GRAPHRAG_FOLDER="/home/yarikama/VGMARS/graph_rag_embedding_experimental/output/20240912-151541/artifacts"

In [2]:
import pandas as pd
from neo4j import GraphDatabase
import time

In [3]:
NEO4J_URI="bolt://localhost"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="reeper30226"
NEO4J_DATABASE="neo4j"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [4]:
def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.
    Parameters: statement is the Cypher query to execute, df is the dataframe to import, and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0,total, batch_size):
        batch = df.iloc[start: min(start+batch_size,total)]
        result = driver.execute_query("UNWIND $rows AS value " + statement, 
                                      rows=batch.to_dict('records'),
                                      database_=NEO4J_DATABASE)
        print(result.summary.counters)
    print(f'{total} rows in { time.time() - start_s} s.')    
    return total

### Indexes and Constraints
Indexes in Neo4j are only used to find the starting points for graph queries, e.g. quickly finding two nodes to connect. Constraints exist to avoid duplicates, we create them mostly on id's of Entity types.

We use some Types as markers with two underscores before and after to distinguish them from the actual entity types.

The default relationship type here is `RELATED` but we could also infer a real relationship-type from the description or the types of the start and end-nodes.

* `__Entity__`
* `__Document__`
* `__Chunk__`
* `__Community__`
* `__Covariate__`

In [7]:
# create constraints, idempotent operation

statements = """
create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique;
create constraint document_id if not exists for (d:__Document__) require d.id is unique;
create constraint entity_id if not exists for (c:__Community__) require c.community is unique;
create constraint entity_id if not exists for (e:__Entity__) require e.id is unique;
create constraint entity_title if not exists for (e:__Entity__) require e.name is unique;
create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique;
create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique;
""".split(";")

for statement in statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)


create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique

create constraint document_id if not exists for (d:__Document__) require d.id is unique

create constraint entity_id if not exists for (c:__Community__) require c.community is unique

create constraint entity_id if not exists for (e:__Entity__) require e.id is unique

create constraint entity_title if not exists for (e:__Entity__) require e.name is unique

create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique

create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique


## Import Process
### Importing the Documents
We're loading the parquet file for the documents and create nodes with their ids and add the title property. We don't need to store text_unit_ids as we can create the relationships and the text content is also contained in the chunks.

In [6]:
doc_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_documents.parquet')
doc_df.head(2)

,id,text_unit_ids,raw_content,title,raw_content_embedding
0,474865e9db94180d0d99e5125ae04f2b,"[a2a6bc7fdeba51d448c87fd4b30ca1e8, 877021065a3...",Rise of the Guardians Script at IMSDb. The Int...,movie_Rise_of_the_Guardians.txt,"[0.016017707094181693, 0.02192370068985526, -0..."
1,dc56f172618f7849cb1cd4075359c4bb,"[fa119fc2d9dd1d9dacc117dfe72c5eb1, f7044ed1e87...",ïProject Gutenbergs The Adventure of the Cardb...,gutenberg_The_Adventure_of_the_Cardboard_Box.txt,"[0.03423983798018732, 0.030299222168653692, 0...."


In [8]:
# import documents
statement = """
MERGE (d:__Document__ {id:value.id})
SET d += value {.title, .raw_content, .raw_content_embedding}
"""

batched_import(statement, doc_df)

{'_contains_updates': True, 'properties_set': 33}
11 rows in 0.057679176330566406 s.


11

### Loading Text Units
We load the text units, create a node per id and set the text and number of tokens. Then we connect them to the documents that we created before.

In [9]:
text_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_text_units.parquet')
text_df.head(2)

,id,text,text_embedding,n_tokens,document_ids,entity_ids,relationship_ids
0,057ff759b344cdcdea9924678f13e0b6,Yes Man Script at IMSDb. The Internet Movie Sc...,"[-0.04536692425608635, 0.08338487148284912, -0...",1024,[0015e9b933802296eb40df4f7a5cd5f3],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[af64651e237b43f2913082095772abd5, d98c4bed659..."
1,9ed083c1a3066c142cb7871b6fa6ded9,saw the ball drop and hugged all\n of you. Al...,"[-0.018350979313254356, 0.050461456179618835, ...",1024,[0015e9b933802296eb40df4f7a5cd5f3],"[4119fd06010c494caa07f439b333f4c5, 077d2820ae1...","[f14fb2a529644d9da08201b47a9c6277, 1a0d8ef2e5b..."


In [10]:
statement = """
MERGE (c:__Chunk__ {id:value.id})
SET c += value {.text, .n_tokens, .text_embedding}
WITH c, value
UNWIND value.document_ids AS document
MATCH (d:__Document__ {id:document})
MERGE (c)-[:PART_OF]->(d)
"""

batched_import(statement, text_df)

{'_contains_updates': True, 'labels_added': 523, 'relationships_created': 523, 'nodes_created': 523, 'properties_set': 2092}
523 rows in 0.7090353965759277 s.


523

### Loading Nodes
For the nodes we store id, name, description, embedding (if available), human readable id.

In [11]:
entity_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_entities.parquet')
entity_df.head(2)

,id,name,type,description,human_readable_id,graph_embedding,text_unit_ids,name_embedding,description_embedding
0,b45241d70f0e43fca764df95b2b81f77,CARL KENDALL,PERSON,Carl Kendall is a multifaceted character navig...,0,None,"[057ff759b344cdcdea9924678f13e0b6, 7464b81026d...","[0.010291694663465023, 0.022835588082671165, -...","[-0.010781682096421719, 0.029624300077557564, ..."
1,4119fd06010c494caa07f439b333f4c5,KATH,PERSON,Kath is a significant character in the narrati...,1,None,"[057ff759b344cdcdea9924678f13e0b6, 21c31cd17da...","[-0.006516867317259312, -0.0052177999168634415...","[-0.014255154877901077, 0.032462652772665024, ..."


In [11]:
entity_statement = """
MERGE (e:__Entity__ {id:value.id})
SET e += value {.human_readable_id, .description, name:replace(value.name,'"','')}
WITH e, value
CALL db.create.setNodeVectorProperty(e, "description_embedding", value.description_embedding)
CALL apoc.create.addLabels(e, case when coalesce(value.type,"") = "" then [] else [apoc.text.upperCamelCase(replace(value.type,'"',''))] end) yield node
UNWIND value.text_unit_ids AS text_unit
MATCH (c:__Chunk__ {id:text_unit})
MERGE (c)-[:HAS_ENTITY]->(e)
"""

batched_import(entity_statement, entity_df)

{'_contains_updates': True, 'labels_added': 1000, 'relationships_created': 4950, 'nodes_created': 1000, 'properties_set': 4000}
{'_contains_updates': True, 'labels_added': 1000, 'relationships_created': 3491, 'nodes_created': 1000, 'properties_set': 4000}
{'_contains_updates': True, 'labels_added': 1000, 'relationships_created': 2926, 'nodes_created': 1000, 'properties_set': 4000}
{'_contains_updates': True, 'labels_added': 1000, 'relationships_created': 3009, 'nodes_created': 1000, 'properties_set': 4000}
{'_contains_updates': True, 'labels_added': 1000, 'relationships_created': 2233, 'nodes_created': 1000, 'properties_set': 4000}
{'_contains_updates': True, 'labels_added': 1000, 'relationships_created': 1363, 'nodes_created': 1000, 'properties_set': 4000}
{'_contains_updates': True, 'labels_added': 1000, 'relationships_created': 2900, 'nodes_created': 1000, 'properties_set': 4000}
{'_contains_updates': True, 'labels_added': 1000, 'relationships_created': 2048, 'nodes_created': 1000, 

16021

### Import Relationships
For the relationships we find the source and target node by name, using the base `__Entity__` type. After creating the RELATED relationships, we set the description as attribute.

In [12]:
rel_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_relationships.parquet',
                         columns=["source","target","id","rank","weight","human_readable_id","description","text_unit_ids"])
rel_df.head(2)

,source,target,id,rank,weight,human_readable_id,description,text_unit_ids
0,IMSDB,MOVIE SCRIPTS,1dfaa8fa5459453f8dd936fca23a7a55,21,1.0,0,IMSDB provides access to a large collection of...,[9ba0038c69e2aa715edc97db9959a835]
1,IMSDB,FILM SCRIPTS,2814c27910dc4fd884d130d096ddb03f,21,1.0,1,"IMSDB hosts a variety of film scripts, making ...",[9ba0038c69e2aa715edc97db9959a835]


In [13]:
rel_statement = """
    MATCH (source:__Entity__ {name:replace(value.source,'"','')})
    MATCH (target:__Entity__ {name:replace(value.target,'"','')})
    // not necessary to merge on id as there is only one relationship per pair
    MERGE (source)-[rel:RELATED {id: value.id}]->(target)
    SET rel += value {.rank, .weight, .human_readable_id, .description, .text_unit_ids}
    RETURN count(*) as createdRels
"""

batched_import(rel_statement, rel_df)

{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_c

25881

### Importing Communities
For communities we import their id, title, level. We connect the `__Community__` nodes to the start and end nodes of the relationships they refer to.

Connecting them to the chunks they orignate from is optional, as the entites are already connected to the chunks.

In [14]:
community_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_communities.parquet', 
                     columns=["id","level","title","text_unit_ids","relationship_ids"])

community_df.head(2)

,id,level,title,text_unit_ids,relationship_ids
0,44,0,Community 44,"[03f118ef6f42b723319dc15b9c04bcdf,08e3f2101016...","[1dfaa8fa5459453f8dd936fca23a7a55, 2814c27910d..."
1,2,0,Community 2,"[03f118ef6f42b723319dc15b9c04bcdf,10a8a3b8dada...","[2488e6594f8848f2965413e3bf42b5f1, 4e2d8f5b037..."


In [15]:
statement = """
MERGE (c:__Community__ {community:value.id})
SET c += value {.level, .title}
/*
UNWIND value.text_unit_ids as text_unit_id
MATCH (t:__Chunk__ {id:text_unit_id})
MERGE (c)-[:HAS_CHUNK]->(t)
WITH distinct c, value
*/
WITH *
UNWIND value.relationship_ids as rel_id
MATCH (start:__Entity__)-[:RELATED {id:rel_id}]->(end:__Entity__)
MERGE (start)-[:IN_COMMUNITY]->(c)
MERGE (end)-[:IN_COMMUNITY]->(c)
RETURN count(distinct c) as createdCommunities
"""

batched_import(statement, community_df)

{'_contains_updates': True, 'labels_added': 1000, 'relationships_created': 58907, 'nodes_created': 1000, 'properties_set': 3000}
{'_contains_updates': True, 'labels_added': 1000, 'relationships_created': 21390, 'nodes_created': 1000, 'properties_set': 3000}
{'_contains_updates': True, 'labels_added': 781, 'relationships_created': 18410, 'nodes_created': 781, 'properties_set': 2343}
2781 rows in 1.610992193222046 s.


2781

### Importing Community Reports
Fo the community reports we create nodes for each communitiy set the id, community, level, title, summary, rank, and rank_explanation and connect them to the entities they are about. For the findings we create the findings in context of the communities.

In [16]:
community_report_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_community_reports.parquet',
                               columns=["id","community","level","title","summary", "findings","rank","rank_explanation","full_content"])
community_report_df.head(2)

,id,community,level,title,summary,findings,rank,rank_explanation,full_content
0,a7d39cf1-f5f1-436b-885d-29f49a5736de,2779,5,Sir Patrick Lundie and Legal Entanglements,The community centers around Sir Patrick Lundi...,[{'explanation': 'Sir Patrick Lundie serves as...,7.5,The impact severity rating is high due to the ...,# Sir Patrick Lundie and Legal Entanglements\n...
1,13efa8fe-e299-4fbd-b008-4392f5f797f2,2780,5,Anne Silvester and Her Complex Relationships,"The community centers around Anne Silvester, a...",[{'explanation': 'Anne Silvester's relationshi...,7.5,The impact severity rating is high due to the ...,# Anne Silvester and Her Complex Relationships...


In [17]:
# import communities
community_statement = """
MERGE (c:__Community__ {community:value.community})
SET c += value {.level, .title, .rank, .rank_explanation, .full_content, .summary}
WITH c, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH c, value, finding_idx, value.findings[finding_idx] as finding
MERGE (c)-[:HAS_FINDING]->(f:Finding {id:finding_idx})
SET f += finding
"""
batched_import(community_statement, community_report_df)

{'_contains_updates': True, 'labels_added': 4898, 'relationships_created': 4898, 'nodes_created': 4898, 'properties_set': 20694}
{'_contains_updates': True, 'labels_added': 4868, 'relationships_created': 4868, 'nodes_created': 4868, 'properties_set': 20604}
{'_contains_updates': True, 'labels_added': 3903, 'relationships_created': 3903, 'nodes_created': 3903, 'properties_set': 16395}
2781 rows in 0.5819230079650879 s.


2781

### Importing Covariates
Covariates are for instance claims on entities, we connect them to the chunks where they originate from.

**By default, covariates are not included in the output, so the file might not exists in your output if you didn't set the configuration to extract claims**

In [21]:
# """
cov_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_covariates.parquet')
cov_df.head(2)
# """

FileNotFoundError: [Errno 2] No such file or directory: '../graph_rag/output/20240825-164936/artifacts/create_final_covariates.parquet'

In [20]:
cov_statement = """
MERGE (c:__Covariate__ {id:value.id})
SET c += apoc.map.clean(value, ["text_unit_id", "document_ids", "n_tokens"], [NULL, ""])
WITH c, value
MATCH (ch:__Chunk__ {id: value.text_unit_id})
MERGE (ch)-[:HAS_COVARIATE]->(c)
"""
batched_import(cov_statement, cov_df)

NameError: name 'cov_df' is not defined